# Record Linkage

In [3]:
import recordlinkage
import pandas as pd

In [4]:
PATH_DS1 = './Record Linkage/Datasets for RL/Disfold/disfold_Avengers_m.json'
PATH_DS2 = './Record Linkage/Datasets for RL/Disfold/disfold_fr_m.json'
THRESHOLD = 0.95

dfA = pd.read_json('./Record Linkage/Datasets for RL/Disfold/disfold_Avengers_m.json')
display(dfA)
dfB = pd.read_json('./Record Linkage/Datasets for RL/Disfold/disfold_fr_m.json')
display(dfB)

,id,name,headquarters,country,continent,region,sub_region,founded,employees,ceo,...,change_1_day,change_1_year,results_for_year,total_assets,total_liabilities,total_equity,founders,market_value,address,type
0,98e1d71b-5bc0-4888-ba3f-88955274837b,Apple,,Cupertino,CA,,,"April 1, 1976","100,000",Mr. Timothy D. Cook,...,,,,,,,,,,
1,d774584e-0d8f-489a-a84b-04287c55c7cb,Danaher,,Washington,DC,,,1969,"78,000",Mr. Rainer M. Blair,...,,,,,,,,,,
2,f1b77b88-4227-49c9-9d89-62af78c52f20,Merck,,Kenilworth,NJ,,,1891,"67,000",Mr. Robert M. Davis J.D.,...,,,,,,,,,,
3,98bdf197-6812-4d08-bf5e-b66ec60ebf62,Bhp,,Melbourne,VIC,,,1885,"40,110","Mr. Mike P. Henry B.Sc., BSc (Chem)",...,,,,,,,,,,
4,6ee7d5ac-7a00-48f2-90c4-ad8756734f16,Nike,,Beaverton,OR,,,"January 25, 1964","73,300",Mr. John J. Donahoe II,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,589309b2-6366-4923-a799-96798ffc0f0c,Asahi Group Holdings Ltd,,Tokyo,Japan,,,,"30,020",Mr. Atsushi Katsuki,...,,,,,,,,,,
996,ffe43cb6-90b7-4f26-904e-8e26105c32b3,Haitong Securities Co. Ltd,,Shanghai,China,,,,"11,282",,...,,,,,,,,,,
997,796a92a2-3c91-43cc-a9af-2386d7e57024,"Catalent, Inc.",,Somerset,NJ,,,2007,"17,300",Mr. John R. Chiminski,...,,,,,,,,,,
998,ecc02541-e061-4fa6-8599-e754a81a871b,"Quanta Services, Inc.",,Houston,TX,,,1997,"43,700",Mr. Earl C. Austin Jr.,...,,,,,,,,,,


,id,name,headquarters,country,continent,region,sub_region,founded,employees,ceo,...,change_1_day,change_1_year,results_for_year,total_assets,total_liabilities,total_equity,founders,market_value,address,type
0,2a50924e-8fea-44ac-9108-d11b5c84db4a,Apple,,,,,,"Founded: April 1, 1976","Employees: 100,000",CEO: Mr. Timothy D. Cook,...,,,,,,,,,,
1,118bc8de-0b90-43fc-b7a4-32835c17448d,Apple.,,,,,,"Founded: April 4, 1975","Employees: 181,000",CEO: Mr. Satya Nadella,...,,,,,,,,,,
2,118bc8de-0b90-43fc-b7a4-32835c17448d,Microsoft,,,,,,"Founded: April 4, 1975","Employees: 181,000",CEO: Mr. Satya Nadella,...,,,,,,,,,,
3,bbc75ca1-e3e4-4d48-858c-59d5cfc0cb80,Aramco,,,,,,,,CEO: Mr. Amin H. Nasser,...,,,,,,,,,,
4,ae47ac0c-74a6-4fde-ae09-53f8f9063dd2,Alphabet,,,,,,"Founded: October 2, 2015","Employees: 156,500",CEO: Mr. Sundar Pichai,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,7cbae98d-b2a0-4c6c-b76b-f6b2f58e950a,Markel Corporation,,,,,,Founded: 1930,"Employees: 20,300",CEO: Mr. Thomas Sinnickson Gayner,...,,,,,,,,,,
896,ee882ec3-4e43-474c-92a1-4e371c8caf18,"Brown & Brown, Inc.",,,,,,Founded: 1939,"Employees: 12,023","CEO: Mr. J. Powell Brown C.P.C.U., CPCU",...,,,,,,,,,,
897,d2e2cb2d-2bcf-4e74-9e13-69b00f04e17f,Nomura Research Institute Ltd,,,,,,,"Employees: 16,196",CEO: Mr. Shingo Konomoto,...,,,,,,,,,,
898,039075cb-7d86-49f5-8304-0db8a85c5c83,MGM Resorts International,,,,,,Founded: 1986,"Employees: 52,000",CEO: Mr. William Joseph Hornbuckle IV,...,,,,,,,,,,


In [5]:
def matches_list_index(path_ds1, path_ds2):
    pairs = []
    dfA = pd.read_json(path_ds1)
    dfB = pd.read_json(path_ds2)
    # set_index
    indexer = recordlinkage.Index()
    indexer.full()
    candidate_links  = indexer.index(dfA, dfB)
    # Comparison step
    compare_cl = recordlinkage.Compare()
    compare_cl.string("name", "name", method="jarowinkler", threshold=THRESHOLD, label="name")
    features = compare_cl.compute(candidate_links, dfA, dfB)
    # Classification step
    matches = features[features.sum(axis=1) >= 0.95]
    # display(matches)
    #df2 = matches[matches.columns[0]]
    di = matches.to_dict()
    for pair in di['name'].keys():
        pairs.append(pair)
    return pairs

def print_row_by_index(path_ds1, i):
    df = pd.read_json(path_ds1)
    display(df.loc[i])


In [8]:
pairs = matches_list_index(PATH_DS1, PATH_DS2)

print(pairs)

indexes_A = indexes_B = []

for x in pairs:
    indexes_A.append(x[0])
    
for x in pairs:
    indexes_B.append(x[1])

# display(indexes_A)
#print(indexes_A, end='')
#print(indexes_B, end='')
# display(pairs[50])


[(0, 0), (0, 1), (1, 50), (2, 49), (4, 47), (5, 46), (6, 45), (7, 44), (8, 43), (10, 41), (11, 40), (12, 39), (14, 37), (16, 35), (17, 34), (18, 33), (19, 32), (20, 31), (22, 29), (24, 27), (25, 26), (26, 25), (27, 24), (28, 23), (29, 22), (30, 21), (31, 20), (33, 18), (34, 17), (36, 15), (37, 14), (38, 13), (39, 12), (40, 11), (42, 9), (43, 8), (44, 7), (45, 6), (46, 5), (47, 4), (48, 3), (49, 2), (50, 78), (51, 77), (52, 76), (54, 74), (55, 73), (57, 71), (58, 70), (59, 69), (60, 68), (61, 67), (62, 66), (63, 65), (65, 63), (67, 61), (69, 59), (71, 57), (72, 56), (73, 55), (74, 54), (75, 53), (76, 52), (77, 51), (78, 100), (80, 98), (81, 97), (84, 94), (87, 91), (89, 89), (91, 87), (92, 86), (93, 85), (94, 84), (95, 83), (98, 80), (99, 79), (100, 150), (101, 149), (103, 147), (104, 146), (105, 145), (106, 144), (107, 143), (108, 142), (109, 141), (110, 140), (111, 139), (112, 138), (113, 137), (115, 135), (116, 134), (117, 133), (118, 132), (119, 131), (122, 128), (123, 127), (124, 1

In [154]:
print_row_by_index(PATH_DS1, 5)

id                                676b47a3-de69-46be-bd4b-ea8d6e48fbea
name                                                            Abbott
headquarters                                                          
country                                                   NorthChicago
continent                                                           IL
region                                                                
sub_region                                                            
founded                                                           1888
employees                                                      113,000
ceo                                                 Mr. Robert B. Ford
market_cap                                                            
categories           [UnitedStates, Healthcare, MedicalDevices, Uni...
sector                                                                
industry                                                              
code  